# Graph-Slam Demonstration

#### The purpose of this notebook is to demonstrate the implementation of a Graph-based approach for solving SLAM
Graph-SLAM formulates the SLAM as an optimisation problem, and is mostly used to solve offline SLAM problems, i.e. optimise all the poses of the robot after the path has been tranversed. This implementation, however is a simplified 2D variant that uses a graph-based approach to perform online estimation. 

GraphSLAM uses the motion information as well as the observations of the environment to create least square problem that can be solved using standard optimization techniques.

In [3]:
import numpy as np
import copy
import math
import itertools
import numpy as np 
import matplotlib.pyplot as plt
from graphSLAM import calc_rotational_matrix, calc_jacobian, cal_observation_sigma, \
                             calc_input, observation, motion_model, Edge, pi_2_pi

%matplotlib inline
np.set_printoptions(precision=3, suppress=True)
np.random.seed(0)

### General Form of Graph-SLAM
The robot is commanded to move forward with a control input $u_t$, however, the motion is not perfect and the measured odometry will deviate from the true path. At each timestep the robot can observe its environment, observing obstacle $i$ at location $(x_i, y_i, z_i)$. The measured observations are the range between the robot and landmark. These measurements are also subjected to noise. 

To solve this, graphSLAM creates what is called as the system information matrix $\Omega$ also referred to as $H$ and the information vector $\xi$ also known as $b$. The entries are created based on the information of the motion and the observation.

Graph Construction

Firstly the nodes are defined $\boldsymbol{x} = \boldsymbol{x}_{1:n}$ such that each node is the pose of the robot at time $t_i$ Secondly, the edges i.e. the constraints, are constructed according to the following conditions:

* robot moves from $\boldsymbol{x}_i$ to $\boldsymbol{x}_j$. This edge corresponds to the odometry measurement. Relative motion constraints (Not included in the previous minimal example).
* Measurement constraints, this can be done in two ways:
    * The information matrix is set in such a way that it includes the landmarks in the map as well. Then the constraints can be entered in a straightforward fashion between a node $\boldsymbol{x}_i$ and some landmark $m_k$
    * Through creating a virtual measurement among all the node that have observed the same landmark. More concretely, robot observes the same landmark from $\boldsymbol{x}_i$ and $\boldsymbol{x}_j$. Relative measurement constraint. The "virtual measurement" $\boldsymbol{z}_{ij}$, which is the estimated pose of $\boldsymbol{x}_j$ as seen from the node $\boldsymbol{x}_i$. The virtual measurement can then be entered in the information matrix and vector in a similar fashion to the motion constraints.

An edge is fully characterized by the values of the error (entry to information vector) and the local information matrix (entry to the system's information vector). The larger the local information matrix (lower $Q$ or $R$) the values that this edge will contribute with.

Important Notes:

* The addition to the information matrix and vector are added to the earlier values.
* In the case of 2D robot, the constraints will be non-linear, therefore, a Jacobian of the error w.r.t the states is needed when updated $H$ and $b$.
* The anchoring constraint is needed in order to avoid having a singular information matri.

### Graph Optimization

The result from this formulation yields an overdetermined system of equations. The goal after constructing the graph is to find: $x^* = \underset{x}{\mathrm{argmin}} ~ \underset{ij} \Sigma ~ f(e_{ij}) $, where $f$ is some error function that depends on the edges between to related nodes $i$ and $j$. The derivation in the references arrive at the solution for $x^* = H^{-1}b$
Planar Example:

Now we will go through an example with a more realistic case of a 2D robot with 3DoF, namely, $[x, y, \theta]^T$